In [4]:
############ tsmixer
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')  # Assuming the module is in the parent directory
import benchmarks
import time
import glob
import os
from preprocess_scripts.data_loader import DataLoader
import tensorflow as tf
import numpy as np
import pandas as pd

# first sort, then do placebo test
def custom_sort_key(s):
    parts = s.split('_')
    return int(parts[1])

def sMAPE_tf(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    smape_values = tf.abs(y_pred - y_true) / (tf.abs(y_pred) + tf.abs(y_true)) * 2
    smape_per_series = tf.reduce_mean(smape_values, axis=1)
    mean_smape = tf.reduce_mean(smape_per_series)

    return mean_smape  # Convert TensorFlow tensor to NumPy array for compatibility


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2024-02-29 10:10:24.642517: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 10:10:24.642593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 10:10:24.644545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 10:10:27.329293: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:


def tsmixer_eval(dataset_name,dataset_type, forecast_horizon):
    if dataset_type == "sim":
        y_true_df_A = pd.read_csv("../datasets/text_data/" + dataset_type +  \
                "/" + dataset_name + "_test_actual.csv")
        # Reading the original data to calculate the MASE errors
        y_true_df_B = pd.read_csv("../datasets/text_data/" + dataset_type +  \
                "/" + dataset_name + "_train.csv")
        data_row_A = y_true_df_A.pivot(index='time', columns='series_id', values='value')
        data_row_B = y_true_df_B.pivot(index='time', columns='series_id', values='value')
        data_row_A = data_row_A.loc[:,sorted(data_row_A.columns, key=custom_sort_key)]
        data_row_B = data_row_B.loc[:,sorted(data_row_B.columns, key=custom_sort_key)]
        data_row = pd.concat([data_row_B, data_row_A],ignore_index=True)
        data_row.to_csv("../datasets/text_data/sim/"+dataset_name+".csv")
        length_of_series = len(data_row.index)

        # The columns of data_row need to be rename
        # Define the pattern to replace
        linear_to_l = r'_linear'
        nonlinear_to_nl = r'_nonlinear'
        heterogeneous_to_he = r'_heterogeneous'
        homogeneous_to_ho = r'_homogeneous'

        # Use regular expression to replace the part in column names
        data_row.columns = data_row.columns.str.replace(linear_to_l, '_l')
        data_row.columns = data_row.columns.str.replace(nonlinear_to_nl, '_nl')
        data_row.columns = data_row.columns.str.replace(heterogeneous_to_he, '_he')
        data_row.columns = data_row.columns.str.replace(homogeneous_to_ho, '_ho')

        data_true_counterfactual = pd.read_csv("../datasets/text_data/sim/"+dataset_name+'_true_counterfactual.csv')
        data_true_counterfactual['time'] = data_true_counterfactual['time']+length_of_series-forecast_horizon-1
        data_true_counterfactual = data_true_counterfactual.pivot(index='time', columns='series_id')['value']
        data_true_counterfactual = data_true_counterfactual.loc[:,sorted(data_true_counterfactual.columns, key=custom_sort_key)]
        # Replace values in data_row using the mapping
        data_row_for_errors = data_row.copy()
        data_row_for_errors.loc[data_true_counterfactual.index, data_true_counterfactual.columns] = data_true_counterfactual
        data_row_for_errors.to_csv("../datasets/text_data/sim/"+dataset_name+"_for_errors.csv")

        data_row_A = data_row_for_errors.iloc[length_of_series-forecast_horizon:, :].T
        data_row_B = data_row_for_errors.iloc[:length_of_series-forecast_horizon, :].T
        
        
    if dataset_type == "calls911":
        control = ["BRIDGEPORT", "BRYN ATHYN", "DOUGLASS", "HATBORO", "HATFIELD BORO",
                      "LOWER FREDERICK", "NEW HANOVER", "NORRISTOWN", "NORTH WALES", "SALFORD",
                      "SPRINGFIELD", "TRAPPE"]
        data_row = pd.read_csv('../datasets/text_data/' + dataset_type\
                            + '/'+dataset_name+'.csv').iloc[:, 1:]
        data_row_cols = data_row.columns
        data_row_for_errors = data_row.loc[:,control]
        length_of_series = len(data_row.index)
        y_true_df_A = data_row_for_errors.iloc[length_of_series-forecast_horizon:, :].T
        y_true_df_B = data_row_for_errors.iloc[:length_of_series-forecast_horizon, :].T
        data_row_A = y_true_df_A
        # print(data_row_A)
        data_row_B = y_true_df_B
    
    feature_type='M'
    norm_type = 'B'
    activation = 'relu'
    dropout = 0.05
    n_block=2
    batch_size=31
    no_of_series = len(data_row_B.index) 
    patience = 5
    train_epochs = 100
    learning_rate = 0.01
    seasonality_period = 12
    
    input_size = int(seasonality_period * 1.25)
    checkpoint_dir = '../checkpoints/'

    data_loader = DataLoader(
        data=dataset_name,
        batch_size=batch_size,
        seq_len=input_size,
        pred_len=forecast_horizon,
        feature_type=feature_type,
        dt_type=dataset_type
    )
    train_data = data_loader.get_train()
    val_data = data_loader.get_val()
    test_data = data_loader.get_test()

    build_model = getattr(benchmarks, 'tsmixer').build_model
    model = build_model(
        input_shape=(input_size, data_loader.n_feature),
        pred_len=forecast_horizon,
        norm_type=norm_type,
        activation=activation,
        dropout=dropout,
        n_block=n_block,
        ff_dim=no_of_series,
        target_slice=data_loader.target_slice,
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss=sMAPE_tf, metrics=[sMAPE_tf])

    checkpoint_path = os.path.join(checkpoint_dir, f'{dataset_name}_best')
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=1,
        save_best_only=True,
        save_weights_only=True,
    )
    early_stop_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=patience
    )
    start_training_time = time.time()
    history = model.fit(
        train_data,
        epochs=train_epochs,
        validation_data=val_data,
        callbacks=[checkpoint_callback, early_stop_callback],
    )
    end_training_time = time.time()
    elasped_training_time = end_training_time - start_training_time
    print(f'Training finished in {elasped_training_time} secconds')

    # evaluate best model
    best_epoch = np.argmin(history.history['val_loss'])
    model.load_weights(checkpoint_path)
    test_result = model.evaluate(test_data)
    test_smape = test_result[1]
    print(test_smape)

    for f in glob.glob(checkpoint_path + '*'):
        os.remove(f)
    
    prediction = model.predict(test_data)
    y_pred = data_loader.inverse_transform(prediction[0])

    output = '../results/benchmarks/predicted/' + dataset_name +\
            '_tsmixer.csv'
    y_pred_df = pd.DataFrame(y_pred.T)
    y_pred_df.to_csv(output, index=False, header=False)
    y_pred_for_errors = y_pred_df.copy()
    if dataset_type == "calls911":
        y_pred_for_errors['names'] = data_row_cols
        y_pred_for_errors.set_index('names', inplace=True)
        y_pred_for_errors = y_pred_for_errors.loc[control,:]

    errors_directory = '../results/benchmarks/errors/'

    errors_file_name_mean_median = 'mean_median_' + dataset_name + '_tsmixer'
    SMAPE_file_name_all_errors = 'all_smape_errors_' + dataset_name + '_tsmixer'
    MASE_file_name_all_errors = 'all_mase_errors_' + dataset_name + '_tsmixer'

    errors_file_full_name_mean_median = errors_directory + errors_file_name_mean_median+'.txt'
    SMAPE_file_full_name_all_errors = errors_directory + SMAPE_file_name_all_errors
    MASE_file_full_name_all_errors = errors_directory + MASE_file_name_all_errors

    # SMAPE
    print(np.array(y_pred_for_errors))
    print(np.array(data_row_A))
    try:
        time_series_wise_SMAPE = 2 * np.abs(np.array(y_pred_for_errors) - np.array(data_row_A)) /\
            (np.abs(np.array(y_pred_for_errors)) + np.abs(np.array(data_row_A)))
        SMAPEPerSeries = np.mean(time_series_wise_SMAPE, axis=1)
        mean_SMAPE = np.mean(SMAPEPerSeries)
        mean_SMAPE_str = f"mean_SMAPE:{mean_SMAPE}"
        print(mean_SMAPE_str)
        np.savetxt(SMAPE_file_full_name_all_errors+'.txt', SMAPEPerSeries, delimiter=",", fmt='%f')
    except:
        print(np.array(y_pred_for_errors))
        print(np.array(data_row_A))
    mase_vector = []
    for i in range(no_of_series):
        lagged_diff = [data_row_B.iloc[i,j] - \
                   data_row_B.iloc[i,j - forecast_horizon]\
                      for j in range(forecast_horizon,\
                        len(data_row_B.columns))]
        mase_vector.append(np.mean(np.abs(np.array(np.array(data_row_A.iloc[i]))\
                 - np.array(y_pred_for_errors.iloc[i])) / np.mean(np.abs(lagged_diff))))

    mean_MASE = np.mean(mase_vector)
    mean_MASE_str = f"mean_MASE:{mean_MASE}"
    print(mean_MASE_str)

    np.savetxt(MASE_file_full_name_all_errors+'.txt', mase_vector, delimiter=",", fmt='%f')

    # Writing the SMAPE results to file
    with open(errors_file_full_name_mean_median, 'w') as f:
        # f.write('\n'.join([mean_SMAPE_str, median_SMAPE_str, std_SMAPE_str]))
        f.write('\n'.join([mean_SMAPE_str]))

    # Writing the MASE results to file
    with open(errors_file_full_name_mean_median, 'a') as f:
        # f.write('\n'.join([mean_MASE_str, median_MASE_str, std_MASE_str]))
        f.write('\n'.join([mean_MASE_str]))


In [10]:
dataset_name = 'calls911_benchmarks'
dataset_type = 'calls911'
forecast_horizon=7

tsmixer_eval(dataset_name,dataset_type,forecast_horizon)

42 22 22
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 1.4638 - sMAPE_tf: 1.4638
Epoch 1: val_loss improved from inf to 1.49645, saving model to ../checkpoints/calls911_benchmarks_best
1/1 [==============================] - 3s 3s/step - loss: 1.4638 - sMAPE_tf: 1.4638 - val_loss: 1.4965 - val_sMAPE_tf: 1.4965
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 1.4514 - sMAPE_tf: 1.4514
Epoch 2: val_loss improved from 1.49645 to 1.47420, saving model to ../checkpoints/calls911_benchmarks_best
1/1 [==============================] - 0s 233ms/step - loss: 1.4514 - sMAPE_tf: 1.4514 - val_loss: 1.4742 - val_sMAPE_tf: 1.4742
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 1.4395 - sMAPE_tf: 1.4395
Epoch 3: val_loss improved from 1.47420 to 1.46167, saving model to ../checkpoints/calls911_benchmarks_best
1/1 [==============================] - 0s 229ms/step - loss: 1.4395 - sMAPE_tf: 1.4395 - val_loss: 1.4617 - val_sMAPE_tf: 1.4617
Epoch 4/

In [15]:
dataset_name_test = ['sim_10_60_l_he', 'sim_10_60_l_ho',\
                     'sim_10_60_nl_he', 'sim_10_60_nl_ho',\
                     'sim_10_222_l_he', 'sim_10_222_l_ho',\
                     'sim_10_222_nl_he', 'sim_10_222_nl_ho',\
                     'sim_101_60_l_he', 'sim_101_60_l_ho',\
                     'sim_101_60_nl_he', 'sim_101_60_nl_ho',\
                     'sim_101_222_l_he', 'sim_101_222_l_ho',\
                     'sim_101_222_nl_he', 'sim_101_222_nl_ho',\
                     'sim_500_60_l_he', 'sim_500_60_l_ho',\
                     'sim_500_60_nl_he', 'sim_500_60_nl_ho',\
                     'sim_500_222_l_he', 'sim_500_222_l_ho',\
                     'sim_500_222_nl_he', 'sim_500_222_nl_ho']
dataset_type = 'sim'
forecast_horizon=12

for i in dataset_name_test:
    print(i)
    tsmixer_eval(i,dataset_type,forecast_horizon)

sim_10_60_l_he
36 27 27
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 1.4197 - sMAPE_tf: 1.4197
Epoch 1: val_loss improved from inf to 1.33398, saving model to ../checkpoints/sim_10_60_l_he_best
1/1 [==============================] - 4s 4s/step - loss: 1.4197 - sMAPE_tf: 1.4197 - val_loss: 1.3340 - val_sMAPE_tf: 1.3340
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 1.3928 - sMAPE_tf: 1.3928
Epoch 2: val_loss improved from 1.33398 to 1.32964, saving model to ../checkpoints/sim_10_60_l_he_best
1/1 [==============================] - 0s 236ms/step - loss: 1.3928 - sMAPE_tf: 1.3928 - val_loss: 1.3296 - val_sMAPE_tf: 1.3296
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 1.3522 - sMAPE_tf: 1.3522
Epoch 3: val_loss improved from 1.32964 to 1.32318, saving model to ../checkpoints/sim_10_60_l_he_best
1/1 [==============================] - 0s 221ms/step - loss: 1.3522 - sMAPE_tf: 1.3522 - val_loss: 1.3232 - val_sMAPE_tf: 1.3232
Epoch 4/

ValueError: Unable to coerce to DataFrame, shape must be (11, 12): given (10, 12)

In [3]:
dataset_name_test = ['sim_10_60_l_he', 'sim_10_60_l_ho',\
                     'sim_10_60_nl_he', 'sim_10_60_nl_ho',\
                     'sim_10_222_l_he', 'sim_10_222_l_ho',\
                     'sim_10_222_nl_he', 'sim_10_222_nl_ho',\
                     'sim_101_60_l_he', 'sim_101_60_l_ho',\
                     'sim_101_60_nl_he', 'sim_101_60_nl_ho',\
                     'sim_101_222_l_he', 'sim_101_222_l_ho',\
                     'sim_101_222_nl_he', 'sim_101_222_nl_ho',\
                     'sim_500_60_l_he', 'sim_500_60_l_ho',\
                     'sim_500_60_nl_he', 'sim_500_60_nl_ho',\
                     'sim_500_222_l_he', 'sim_500_222_l_ho',\
                     'sim_500_222_nl_he', 'sim_500_222_nl_ho']
dataset_type = 'sim'
forecast_horizon=12

for i in dataset_name_test:
    print(i)
    tsmixer_eval(i,dataset_type,forecast_horizon)

sim_10_60_l_he
35 27 27
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 1.5557 - sMAPE_tf: 1.5557
Epoch 1: val_loss improved from inf to 1.42770, saving model to ./checkpoints/sim_10_60_l_he_best
1/1 [==============================] - 5s 5s/step - loss: 1.5557 - sMAPE_tf: 1.5557 - val_loss: 1.4277 - val_sMAPE_tf: 1.4277
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 1.5246 - sMAPE_tf: 1.5246
Epoch 2: val_loss did not improve from 1.42770
1/1 [==============================] - 0s 236ms/step - loss: 1.5246 - sMAPE_tf: 1.5246 - val_loss: 1.4536 - val_sMAPE_tf: 1.4536
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 1.5050 - sMAPE_tf: 1.5050
Epoch 3: val_loss did not improve from 1.42770
1/1 [==============================] - 0s 234ms/step - loss: 1.5050 - sMAPE_tf: 1.5050 - val_loss: 1.4632 - val_sMAPE_tf: 1.4632
Epoch 4/100
1/1 [==============================] - ETA: 0s - loss: 1.4967 - sMAPE_tf: 1.4967
Epoch 4: val_loss did not im